In [1]:
import requests
from requests.structures import CaseInsensitiveDict
import json
import re

all_news_titles = []

# 第一次遇见JSONP？数据让我一顿好找
urls = ["https://tech.163.com/special/00097UHL/tech_datalist.js?callback=data_callback", "https://tech.163.com/special/00097UHL/tech_datalist_02.js?callback=data_callback", "https://tech.163.com/special/00097UHL/tech_datalist_03.js?callback=data_callback"]

headers = CaseInsensitiveDict()
headers["Referer"] = "https://tech.163.com/"
headers["user-agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"

for url in urls:
    resp = requests.get(url, headers=headers)
    if resp.status_code == 200:
        # 清除JSONP回调函数
        json_data = resp.text[len('data_callback('):-1]
        json_data = json.loads(json_data)

        news_titles = [item['title'] for item in json_data]
        all_news_titles.extend(news_titles)


# xpath helper "//div[@class='news_title']/h3/a" 出来应该是90个标题
print(len(all_news_titles))
all_news_titles

90


['中国面向全球开放12个核科研设施',
 '最快明年上市！小米SUV会是法拉利Purosangue的平替？',
 '刚摸完苹果iPhone 16系列和手表耳机新品，真机和看图真不一样',
 '头版头条：我国首次！给超级月亮拍了张X光片',
 'iPhone 16 Pro首发评测解禁：5倍长焦被国产旗舰碾压 鬼影依旧',
 '看完这30多个新功能，再来决定要不要升级iOS18。',
 '最快明年上市！小米SUV会是法拉利Purosangue的平替？',
 '让雷军连发N条微博庆祝，啥测试这么有含金量？',
 '云盘惊现陌生人照片?阿里云又双叒叕“翻车”了',
 '9月18日外媒科学网站摘要：如何能发现氢原子大小黑洞穿过太阳系',
 '最多融资1000亿美元！贝莱德和微软联手建AI基础设施',
 'SpaceX被美监管部门罚款63万美元，马斯克威胁反诉',
 '抖音变长，长过B站',
 '小鹏回应MONA M03自动泊车撞上奥迪：不排除光线等因素致弱势场景',
 '台积电美国厂开始生产苹果A16 良率接近台湾厂水平',
 '字节跳动回应与台积电合作AI芯片：报道不实',
 '马斯克脑机接口公司“盲视”设备获“突破性设备认定”',
 'SpaceX为欧洲发射两颗伽利略导航卫星，完成今年第91次发射任务',
 '智行R7将开启大定 余承东“幸福的烦恼”：车太多，展厅没地方放',
 '亚马逊将终止弹性工作政策，要求员工每周5天在办公室工作',
 '英特尔推迟在德国、波兰建芯片工厂，欧洲芯片制造业发展计划受冲击',
 'SpaceX与美联航签约，星链空中WiFi订单翻倍至2500架飞机',
 '未来近2个月，天上将有“两个月亮”？天文学家揭示真相',
 '福特与林肯在中国的焦虑，写在了吉姆·法利的脸上',
 '“霉霉”的这个决定，牵动着汽车产业投资者的神经',
 '理想首款纯电SUV谍照曝光：尾部重新设计',
 '超百名印度工人被捕，三星工厂罢工给“印度制造”蒙阴影',
 '滴滴出行数据报告：中秋节打车上涨43%',
 '亚马逊云科技：Amazon EC2 P5e 实例正式可用',
 '联想发布报告：承诺2049/50财年实现全价值链净零排放总目标',
 '喜茶内部信：行业陷入了同质化竞争 我们不做低价内卷',
 '“想在电动车领域迎头赶上，底特律需邀请中国加入”',
 ' 

In [2]:
# 跟着视频去重 虽然理论上这样取出来没有重复标题
all_news_titles = list(set(all_news_titles))

keyword = 'AI'
keyword_lower = keyword.lower()
cnt = 0
result = []

for title in all_news_titles:
    title_lower = title.lower()
    if keyword_lower in title_lower:
        # 记录标题index 以及 出现次数 暂且把出现次数当成得分。。
        result.append([cnt, title_lower.count(keyword_lower)])
    cnt+=1

result.sort(key=lambda x: x[1], reverse=True)

for item in result:
    # 1 用f string创建一个捕获组()
    # 2 \1引用捕获组 有\所以需要r
    highlighted_title = re.sub(pattern=f'({keyword})', repl=r'*\1*', string=all_news_titles[item[0]], flags=re.IGNORECASE)
    print('[{}] {}'.format(item[1], highlighted_title))

[2] 扩大*AI*版图！报道：英伟达计划以1.65亿美元收购初创公司Octo*AI*
[2] Open*AI* o1开辟“慢思考”，国产*AI*早已集结在CoE“组团”先出发
[1] *AI* 成功进军好莱坞，视频生成平台 Runway 与狮门影业达成合作
[1] 针对全球零工招聘的*AI*助手来了！基于GPT-4构建，自动化程度大幅提升
[1] 最多融资1000亿美元！贝莱德和微软联手建*AI*基础设施
[1] 谷歌论文提前揭示o1模型原理：*AI*大模型竞争或转向硬件
[1] 谷歌将采用C2PA认证判定图片是否*AI*生成
[1] 300亿美元！微软携手贝莱德成立全球最大*AI*投资基金
[1] 美国加州再推五项新法案以加强*AI*监管，但最受争议的一项仍悬而未决
[1] YouTube 将推出 *AI*“一站式服务”：可生成创意、标题、完整视频
[1] Open*AI*押注的「1X」训出专用世界模型，首证机器人Scaling Law
[1] 行研精选丨商业航天市场前景再拓宽；*AI*大模型迈入新阶段
[1] 字节跳动回应与台积电合作*AI*芯片：报道不实
[1] 微软联合贝莱德推300亿美元*AI*基金，将投数据中心和能源
